In [ ]:
import pandas as pd
import numpy as np
import scipy
import nltk
import spacy
import gensim
import glob
import csv
from spellchecker import SpellChecker
import matplotlib
import matplotlib.pyplot as plt
import sklearn
from sklearn.model_selection import cross_val_score
import sklearn.model_selection
import sklearn.pipeline
import re
from sklearn import svm
from sklearn import *
from sklearn.feature_selection import SelectKBest, VarianceThreshold
from sklearn.feature_selection import chi2
from sklearn.base import BaseEstimator, TransformerMixin
import gensim.models.wrappers.fasttext
from scipy import sparse
import tensorflow_datasets as tfds
import tensorflow as tf
import collections
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import LeaveOneOut,KFold,train_test_split
import ktrain
from ktrain import text
from sklearn.metrics import accuracy_score
import simplejson
import pprint


# Custom imports
from mr_generic_scripts import *
from mr_cls_Transformer import *
from mr_generic_scripts import load_combined_data


In [ ]:
# Path to xlsx files folder

# Original MIND-CA corpus
path_to_raw_files = f_path + 'Data/raw_relabeled/'
# MIND-CA + human augment
path_to_plus_files = f_path + 'Data/raw_plus/'
# UK-MIND-20
path_to_wp1 = f_path + 'Data/wp1/'

# Augmented data
# augmentations, 125 examples per QA pair
path_to_aug = f_path + 'Data/aug_data/all/'
# augmentations, 500 examples per QA pair
path_to_aug_hq_os = f_path + 'Data/aug_data_os/all/'
# augmentation, no sampling - 1500 total examples per question
path_to_aug_joint = f_path + 'Data/aug_data_joint/all/'

# Merged xlsx files with multiple augmentations
path_to_set_files = f_path + 'Data/aug_data/sets/'

In [ ]:
# List of augmentations by category
hq_data = ['reord','phrase','dict']
lq_data = ['wordnet','ppdb','glove','fasttext']
set_data = ['ab_lq','ab_hq','all_lq','all_hq','all_aug']

In [ ]:
# General config of the training run (!)

# List of data to use for training

# All possible train sets
# train_sets = ['orig','plus','reord','phrase','dict','wordnet','ppdb','glove','fasttext','ab_lq','ab_hq','all_lq','all_hq']

# Selective train set
#train_sets = ['reord','phrase','dict']
train_sets = ['wordnet','ppdb','glove','fasttext']
#train_sets = ['wp1']

# Alias path to aug data (either 125 or 500 examples or the 1500 joint)
aug_path = path_to_aug_joint

# Training parameters
# Number of folds for k-fold cross validation
n_k_fold = 10

# Only answers (False) or questions + answers (True)
mind_qa = True

In [ ]:
# Load the combined dataset (original + augmented) for training and testing
combined_data = load_combined_data()



# # Get the datasets in dataframes
# datasets = {}

# # Check if we load only answers or questions plus answers
# if mind_qa:
#     # Always load MIND-CA + human aug, this is the base set
#     datasets['plus'] = mr_get_qa_data(path_to_plus_files)

#     # Always load UK-MIND-20, we need it for testing
#     datasets['wp1'] = mr_get_qa_data(path_to_wp1)

#     # If comparison is needed, load MIND-CA without any aug
#     if 'orig' in train_sets:
#         datasets['orig'] = mr_get_qa_data(path_to_raw_files)

#     # Load augmented data
#     for at_set in train_sets:
#         if at_set in ['orig','plus','wp1']:
#             continue
#         path_to_aug = aug_path + at_set + "/"
#         datasets[at_set] = mr_get_qa_data(path_to_aug)

    
# # Only the answer
# else:
#     # Always load MIND-CA + human aug, this is the base set
#     datasets['plus'] = mr_get_data(path_to_plus_files)

#     # Always load UK-MIND-20, we need it for testing
#     datasets['wp1'] = mr_get_data(path_to_wp1)

#     # If comparison is needed, load MIND-CA without any aug
#     if 'orig' in train_sets:
#         datasets['orig'] = mr_get_data(path_to_raw_files)

#     # Load augmented data
#     aug_dataset = {}
#     for at_set in train_sets:
#         if at_set in ['orig','plus','wp1']:
#             continue
#         path_to_aug = aug_path + at_set + "/"
#         datasets[at_set] = mr_get_data(path_to_aug)

In [ ]:
# # Sanity check
# for d_id in train_sets:
#     print(len(datasets[d_id][-1][1]))
#     if at_set in ['orig','plus','wp1']:
#          continue
#     # Augmented datasets have additional column that needs to be dropped
#     datasets[d_id][-1][1].drop(["Aug_ID"],axis=1,inplace=True)

In [ ]:
def mr_proc_results(raw_results):

  pr_results = [[[acc_score, f1_score,round(sum(qf_s)/11,2)],[acc_score_wp1, f1_score_wp1,round(sum(qf_s_wp1)/11,2)]] 
                for ([acc_score, qa_s, aa_s], [f1_score, qf_s, af_s],
                     [acc_score_wp1, qa_s_wp1, aa_s_wp1], [f1_score_wp1, qf_s_wp1, af_s_wp1]) in raw_results]

  # Throw the list in an np array
  pr_arr = np.array(pr_results)

  # Print the results
  pp = pprint.PrettyPrinter(indent=4)

  pp.pprint(pr_results)
  pp.pprint(np.mean(pr_arr,axis=0))

In [ ]:
# Initialize the classifier
if mind_qa:
  # ages 8 to 13, removing outliers; allowed classes 0,1,2; max len 35
  tr_cls = MR_transformer(text_cols,[8,9,10,11,12,13],[0,1,2],35)
else:
  # ages 8 to 13, removing outliers; allowed classes 0,1,2; max len 20
  tr_cls = MR_transformer(text_cols,[8,9,10,11,12,13],[0,1,2],20)


tr_cls.mr_set_eval_vars(True,True,False)

In [ ]:
# Initialize the results variable 
results = {}

In [ ]:
# Split the combined data into training and validation sets
train_df, test_df = train_test_split(combined_data, test_size=0.2)

# Use k-fold cross-validation on the combined dataset
results = tr_cls.mr_kfold_pre_split(train_df, test_df, 0.25, n_k_fold)

# Save the results for the combined dataset
rs_path = 'Results/split_eval_joint/tr_qa_combined.txt'
with open(rs_path, 'w') as op:
    simplejson.dump(results, op)





In [ ]:
# Visualize the results for the combined dataset
print("Combined dataset results:")
mr_proc_results(results)


